# 0. Dependencies


In [ ]:
import os
import bs4
import math
import json
import requests
import itertools

import regex as re
import numpy as np
import pandas as pd

from datetime import date
from getpass import getpass
from bs4 import BeautifulSoup
from xgboost import XGBRegressor

from dateutil.relativedelta import relativedelta
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
# Obtendo o token 'x-funds-nonce'
token_url = 'https://www.fundsexplorer.com.br/wp-content/themes/fundsexplorer/dist/frontend.min.js'
response = requests.get(token_url)
funds_explorer_token = re.search(r'(?<=\"x-funds-nonce\":\s*\").*?(?=\")', response.text).group()

In [ ]:
# Variáveis globais
start_date = '01-01-2021'
end_date = '01-01-2024'

# 1. FIIs listados na B3

In [ ]:
def get_all_funds():

    # Fazendo requisição à API com o token obtido
    url = 'https://www.fundsexplorer.com.br/wp-json/funds/v1/get-ranking'
    response = requests.get(url, headers={
        'accept': 'application/json, text/plain, */*',
        'x-funds-nonce': funds_explorer_token
    })

    # Convertendo a resposta JSON para DataFrame
    df = pd.read_json(response.json())

    return df

# Exibindo o DataFrame
df_funds = get_all_funds()
df_funds

,post_id,ticker,dividendo,yeld,media_yield_3m,soma_yield_3m,media_yield_6m,soma_yield_6m,media_yield_12m,soma_yield_12m,...,patrimonio,pvp,p_vpa,post_title,ativos,volatility,numero_cotista,tx_gestao,tx_admin,tx_performance
0,2555,AAGR11,1.0636,1.07,1.1400,3.42,1.3850,5.54,1.3850,5.54,...,0.000000e+00,0.0000,NaN,AAGR11,,58.782301975493,0,,,
1,495,AAZQ11,0.1100,1.47,1.3067,3.92,1.3033,7.82,1.3008,15.61,...,2.086440e+08,0.8940,0.894009,AAZQ11,,17.764757056389,27954,,,
2,238,ABCP11,0.3500,0.50,0.6100,1.83,0.6950,4.17,0.7600,9.12,...,4.778136e+08,0.6642,0.664236,ABCP11,1,14.923337745314,16357,,,
3,177,AFHI11,0.9600,1.00,0.9833,2.95,0.9833,5.90,1.0317,12.38,...,3.839856e+08,1.0174,1.017383,AFHI11,0,6.91653246067,38421,,,
4,553,AGRX11,0.1300,1.33,1.3467,4.04,1.2617,7.57,1.2983,15.58,...,0.000000e+00,0.0000,NaN,AGRX11,0,15.638239844426,0,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,537,YUFI11B,0.6331,0.00,0.0000,0.00,0.0000,0.00,0.0000,0.00,...,0.000000e+00,0.0000,NaN,YUFI11B,,,0,,,
478,3486,ZAGH11,0.1100,0.92,0.3067,0.92,0.3067,0.92,0.3067,0.92,...,4.667184e+07,0.1195,NaN,ZAGH11,,390.42346644832,120,,,
479,2972,ZAVC11,0.1100,0.00,0.0000,0.00,0.0000,0.00,0.0000,0.00,...,3.083557e+07,1.0050,NaN,ZAVC11,,8.340380435713,84,,,
480,1866,ZAVI11,1.0700,0.94,0.9433,2.83,0.9733,5.84,0.9492,11.39,...,1.555315e+08,0.7693,NaN,ZAVI11,11,27.066004538363,4392,,,


# 2. Dividendos pagos

In [ ]:
def get_dividends(ticker_list):
    result = pd.DataFrame()

    for index, ticker in enumerate(ticker_list):
        print(f'Obtendo dividendos: {index+1}/{len(ticker_list)}')

        url = f'https://www.fundsexplorer.com.br/wp-json/funds/v1/fund/{ticker}/incomes'

        headers = {
            'accept': 'application/json, text/plain, */*',
            'x-funds-nonce': funds_explorer_token
        }

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            df = pd.read_json(response.json())
            result = pd.concat([result, df], ignore_index=True)
        else:
            print(f"Não foi possível obter os dividendos para o ticker {ticker}.")

    return result

df_dividends = get_dividends(list(df_funds['ticker']))

Obtendo dividendos: 1/14
Obtendo dividendos: 2/14
Obtendo dividendos: 3/14
Obtendo dividendos: 4/14
Obtendo dividendos: 5/14
Obtendo dividendos: 6/14
Obtendo dividendos: 7/14
Obtendo dividendos: 8/14
Obtendo dividendos: 9/14
Obtendo dividendos: 10/14
Obtendo dividendos: 11/14
Obtendo dividendos: 12/14
Obtendo dividendos: 13/14
Obtendo dividendos: 14/14


In [ ]:
df_dividends

,id,ticker,created_at,updated_at,valor,tipo,data_pagamento,data_base,yeld,rentabilidade,...,mes,ano,rentabilidade_mes,media_yield_3m,media_yield_6m,media_yield_12m,soma_yield_3m,soma_yield_6m,soma_yield_12m,soma_yield_ano_corrente
0,8126516,BCFF11,2024-07-12 19:13:42,2024-07-12 19:13:42,0.0700,Rendimento,2024-07-12,2024-07-05,0.83,-1.5432,...,7,2024,-1.5122,0.8067,0.8033,0.7858,2.42,4.82,9.43,5.58
1,8126515,BCFF11,2024-07-12 19:13:42,2024-07-12 19:13:42,0.0700,Rendimento,2024-06-14,2024-06-07,0.81,-0.0315,...,6,2024,-3.7723,0.8067,0.7917,0.7817,2.42,4.75,9.38,4.75
2,8126514,BCFF11,2024-07-12 19:13:42,2024-07-12 19:13:42,0.0700,Rendimento,2024-05-15,2024-05-08,0.78,3.8874,...,5,2024,-1.7259,0.8033,0.7867,0.7817,2.41,4.72,9.38,3.94
3,8126513,BCFF11,2024-07-12 19:13:42,2024-07-12 19:13:42,0.0770,Rendimento,2024-04-12,2024-04-05,0.83,5.7119,...,4,2024,3.2865,0.8000,0.7900,0.7858,2.40,4.74,9.43,3.16
4,8126512,BCFF11,2024-07-12 19:13:42,2024-07-12 19:13:42,0.0725,Rendimento,2024-03-14,2024-03-07,0.80,2.3481,...,3,2024,0.4662,0.7767,0.7817,0.7917,2.33,4.69,9.50,2.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167,8115784,XPML11,2024-07-12 19:07:55,2024-07-12 19:07:55,0.4955,Rendimento,2018-06-25,2018-06-18,0.00,NaN,...,6,2018,0.0000,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00
1168,8115783,XPML11,2024-07-12 19:07:55,2024-07-12 19:07:55,0.5269,Rendimento,2018-05-25,2018-05-18,0.00,NaN,...,5,2018,0.0000,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00
1169,8115782,XPML11,2024-07-12 19:07:55,2024-07-12 19:07:55,0.5028,Rendimento,2018-04-25,2018-04-18,0.00,NaN,...,4,2018,0.0000,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00
1170,8115781,XPML11,2024-07-12 19:07:55,2024-07-12 19:07:55,0.4951,Rendimento,2018-03-23,2018-03-16,0.00,NaN,...,3,2018,0.0000,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00


In [ ]:
df_dividends

,id,ticker,created_at,updated_at,valor,tipo,data_pagamento,data_base,yeld,rentabilidade,...,mes,ano,rentabilidade_mes,media_yield_3m,media_yield_6m,media_yield_12m,soma_yield_3m,soma_yield_6m,soma_yield_12m,soma_yield_ano_corrente
0,8126516,BCFF11,2024-07-12 19:13:42,2024-07-12 19:13:42,0.0700,Rendimento,2024-07-12,2024-07-05,0.83,-1.5432,...,7,2024,-1.5122,0.8067,0.8033,0.7858,2.42,4.82,9.43,5.58
1,8126515,BCFF11,2024-07-12 19:13:42,2024-07-12 19:13:42,0.0700,Rendimento,2024-06-14,2024-06-07,0.81,-0.0315,...,6,2024,-3.7723,0.8067,0.7917,0.7817,2.42,4.75,9.38,4.75
2,8126514,BCFF11,2024-07-12 19:13:42,2024-07-12 19:13:42,0.0700,Rendimento,2024-05-15,2024-05-08,0.78,3.8874,...,5,2024,-1.7259,0.8033,0.7867,0.7817,2.41,4.72,9.38,3.94
3,8126513,BCFF11,2024-07-12 19:13:42,2024-07-12 19:13:42,0.0770,Rendimento,2024-04-12,2024-04-05,0.83,5.7119,...,4,2024,3.2865,0.8000,0.7900,0.7858,2.40,4.74,9.43,3.16
4,8126512,BCFF11,2024-07-12 19:13:42,2024-07-12 19:13:42,0.0725,Rendimento,2024-03-14,2024-03-07,0.80,2.3481,...,3,2024,0.4662,0.7767,0.7817,0.7917,2.33,4.69,9.50,2.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167,8115784,XPML11,2024-07-12 19:07:55,2024-07-12 19:07:55,0.4955,Rendimento,2018-06-25,2018-06-18,0.00,NaN,...,6,2018,0.0000,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00
1168,8115783,XPML11,2024-07-12 19:07:55,2024-07-12 19:07:55,0.5269,Rendimento,2018-05-25,2018-05-18,0.00,NaN,...,5,2018,0.0000,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00
1169,8115782,XPML11,2024-07-12 19:07:55,2024-07-12 19:07:55,0.5028,Rendimento,2018-04-25,2018-04-18,0.00,NaN,...,4,2018,0.0000,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00
1170,8115781,XPML11,2024-07-12 19:07:55,2024-07-12 19:07:55,0.4951,Rendimento,2018-03-23,2018-03-16,0.00,NaN,...,3,2018,0.0000,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00


# 3. Preços de fechamento

In [ ]:
def get_close(ticker_list):

    result = pd.DataFrame()

    for index, ticker in enumerate(ticker_list):

        # print(f'Obtendo preços de fechamento: {index+1}/{len(ticker_list)}')

        response = requests.get(f'https://fii-api.infomoney.com.br/api/v1/fii/cotacao/historico/grafico?Ticker={ticker}&DataInicio={start_date}&DataFim={end_date}')

        if not str(response.content) == "b''":

            json_response = json.loads(response.content)

            if 'errors' in json_response or response.status_code != 200:
                print(f"Não foi possível obter os preços de fechamento para o ticker {ticker}.")
            else:
                df = pd.read_json(json.dumps(json_response['dataValor']))
                df['ticker'] = ticker
                result = pd.concat([result, df], ignore_index=True)

        else:
            print(f"Não foi possível obter os preços de fechamento para o ticker {ticker}.")

    return result

df_close = get_close(list(df_funds['ticker']))

# 4. Endereços dos imóveis dos FIIs

In [ ]:
def get_address(ticker_list):
    result = pd.DataFrame()

    for index, ticker in enumerate(ticker_list):
        # print(f'Obtendo endereços: {index+1}/{len(ticker_list)}')

        api_url = f'https://fii-api.infomoney.com.br/api/v1/propertie/{ticker}'
        response = requests.get(api_url)
        data = []

        if '{' in str(response.content):
            response = response.json()

            for item in response["property"]:
                row = {
                    "Ticker": ticker,
                    "Tipo": item["type"],
                    "Nome": item["name"],
                    "DataCompra": item["datePurchase"],
                    "ValorAreaBrutaLocavel": item["valueGrossLeasableArea"],
                    "Estado": item["state"],
                    "Cidade": item["city"],
                    "Endereco": item["address"],
                    "GoogleMapsLink": item["googleMapsLink"],
                    "PercentualPartic": item["percentagePartic"],
                    "PecentualVacancia": item["percentVacancy"],
                    "PercentualInadimplencia90Dias": item["percent90DayDeliquency"],
                    "PercentualFii": item["percentFii"],
                    "Latitude": float("NaN"),
                    "Longitude": float("NaN")
                }

                # Verifica se as coordenadas estão presentes no link do Google Maps
                cordinates = re.findall("(?<=@)[-]*[\d.]*,-[\d.]*", item['googleMapsLink'])

                if(len(cordinates) > 0):
                    cordinates = cordinates[0].split(",")
                    row["Latitude"], row["Longitude"] = float(cordinates[0]), float(cordinates[1])
                else:
                    # Se não estiverem presentes, faz uma solicitação à página do Google Maps para obter as coordenadas
                    address_url = ("https://www.google.com/maps/place/" + item["address"] + "," + item["city"] + "-" + item["state"]).replace(" ", "%20")
                    response = requests.get(address_url)

                    cordinates = re.findall("(?<=@)[-]*[\d.]*,-[\d.]*", str(response.content))

                    if(len(cordinates) > 0):
                        cordinates = cordinates[0].split(",")
                        row["Latitude"], row["Longitude"] = float(cordinates[0]), float(cordinates[1])

                data.append(row)

            df = pd.DataFrame(data)
            result = pd.concat([result, df], ignore_index=True)

    return result

df_address = get_address(list(df_funds['ticker']))

In [ ]:
percent = df_address['Latitude'].isnull().sum()/(len(df_address))*100
print(f'{len(df_address)} endereços de fundos imobiliários foram encontrados.')
print('%.2f%% dos endereços estão sem Latitude e Longitude.' % percent)

167 endereços de fundos imobiliários foram encontrados.
4.79% dos endereços estão sem Latitude e Longitude.


# 5. Índice IFIX

In [ ]:
def get_ifix():

    df_ifix = pd.DataFrame()

    headers = {
        'authority':'www.infomoney.com.br',
        'accept':'application/json, text/javascript, */*; q=0.01',
        'accept-language':'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
        'content-type':'application/x-www-form-urlencoded; charset=UTF-8',
        'origin':'https://www.infomoney.com.br',
        'referer':'https://www.infomoney.com.br/cotacoes/b3/indice/ifix/historico/',
    }

    body = f'page=0&numberItems=99999&initialDate={start_date}&finalDate={end_date}&symbol=IFIX'
    response = requests.post('https://www.infomoney.com.br/wp-json/infomoney/v1/quotes/history', headers=headers, data=body)

    if not str(response.content) == "b''":

        json_response = json.loads(response.content)

        jobject = []
        for obj in json_response:
            jobject.append({
                'data': obj[0]['display'],
                'Close': obj[2]
            })

        df_ifix = pd.DataFrame(jobject)
        df_ifix['Datetime'] = pd.to_datetime(df_ifix['data'], format='%d/%m/%Y')
        df_ifix.drop(columns={'data'}, inplace = True)

    return df_ifix

df_ifix = get_ifix()
df_ifix

,Close,Datetime
0,3.311,2023-12-28
1,3.299,2023-12-27
2,3.285,2023-12-26
3,3.260,2023-12-22
4,3.232,2023-12-21
...,...,...
493,2.760,2022-01-07
494,2.757,2022-01-06
495,2.760,2022-01-05
496,2.787,2022-01-04


# 6. Índices IGP-M, Selic e IPCA

In [ ]:
def obtem_dados_mercado():

    result = pd.DataFrame()
    indices = ['selic', 'ipca', 'igp-m']

    for indice in indices:

        # Obtém os dados do IGP-M
        response = requests.get(f'https://www.dadosdemercado.com.br/indices/{indice}')

        if response.status_code == 200:
            df = pd.read_html(response.content, encoding='utf-8')[0]

        anos = list(df.iloc[:, 0].values)

        timestamp = []
        values = []

        for i in range(len(anos)):
            for m in range(12, 0, -1):
                taxa = str(list(df.iloc[:, m].values)[i])
                if taxa != '--':
                    mes = str(m) if m > 9 else "0" + str(m)
                    timestamp.append(str(anos[i]) + "-" + mes)
                    values.append(round(float(taxa.replace("%","").replace(",",".")), 2))

        # Cria DataFrame
        df = pd.DataFrame({
            'Timestamp': timestamp,
            'Value': values,
            'Indice': indice.upper()
        })

        df['Value'] = pd.to_numeric(df['Value'], downcast="float")
        result = pd.concat([result, df], ignore_index=True)

    return result

# Exemplo de uso
df_indices = obtem_dados_mercado()
df_indices

,Timestamp,Value,Indice
0,2024-07,10.40,SELIC
1,2024-06,10.40,SELIC
2,2024-05,10.65,SELIC
3,2024-04,10.65,SELIC
4,2024-03,11.15,SELIC
...,...,...,...
878,2000-05,0.31,IGP-M
879,2000-04,0.23,IGP-M
880,2000-03,0.15,IGP-M
881,2000-02,0.35,IGP-M


# 7. Desdobramentos de FIIs

In [ ]:
# Função para converter o Ratio
def convert_ratio(ratio):
    left, right = ratio.split(':')
    return float(left) / float(right)

def get_splits():
    url = "https://br.investing.com/stock-split-calendar/Service/getCalendarFilteredData"
    headers = {
        "accept-language": "en-US,en;q=0.9,pt-BR;q=0.8,pt;q=0.7,es;q=0.6,ar;q=0.5",
        "content-type": "application/x-www-form-urlencoded",
        "origin": "https://br.investing.com",
        "referer": "https://br.investing.com/stock-split-calendar/",
        "x-requested-with": "XMLHttpRequest"
    }
    data = {
        "country[]": "32",
        "dateFrom": "2019-01-01",
        "dateTo": "2024-01-01",
        "currentTab": "custom",
        "submitFilters": "1",
        "limit_from": "0"
    }

    response = requests.post(url, headers=headers, data=data)

    matches = re.finditer(r'\d{2}\.\d{2}\.\d{4}(.|\n)*?(?=\d{2}\.\d{2}\.\d{4}|$)', response.json()['data'])

    df_splits = pd.DataFrame()
    for match in matches:

        match_text = match.group(0)

        date = re.search(r'^(\d{2})\.(\d{2})\.(\d{4})', match_text)

        if date:

            matches2 = re.findall(r'(\d?[A-Z]{4}11B?)(?:.|\n)*?(\d+\:\d+)', match_text)
            df = pd.DataFrame(matches2, columns=['Ticker', 'Ratio'])
            df['Date'] = f'{date.group(3)}-{date.group(2)}-{date.group(1)}'
            df_splits = pd.concat([df_splits, df], ignore_index=True)

    df_splits['Ratio'] = df_splits['Ratio'].apply(convert_ratio)
    df_splits['Date'] = pd.to_datetime(df_splits['Date'])

    return df_splits

df_splits = get_splits()
df_splits

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# 8. Enriquecimento de dados de imóveis - Google Maps

In [ ]:
def get_haversine_calculus(lat1, lon1, lat2, lon2):
    # Converter graus para radianos
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Raio médio da Terra em quilômetros
    radius = 6371

    # Diferença das coordenadas
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Fórmula de Haversine
    a = math.sin(dlat/2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = radius * c

    return distance * 1000

def convert_to_meters(valor):
    if valor.endswith('km'):
        valor = float(valor[:-2]) * 1000
    elif valor.endswith('m'):
        valor = float(valor[:-1])
    return str(int(valor))

def get_walking_distance(lat_origem, lon_origem, lat_destino, lon_destino):

    url = f"https://www.google.com/maps/dir/'{lat_destino},{lon_destino}'/{lat_origem},{lon_origem}/@{lat_destino},{lon_destino},17z/data=!3m1!4b1!4m7!4m6!1m3!2m2!1d{lon_destino}!2d{lat_destino}!1m0!3e2?entry=ttu"

    response = requests.get(url)
    content = response.content.decode('utf-8')

    pattern = r'\[[\d.]+,\\"([\d.]+\s+(?:km|m))\\",[\d.]+\],\[[\d.]+,\\"[\d.]+\s+(?:min|sec|hr)'
    result = re.findall(pattern, content)

    if result:
        return convert_to_meters(result[0])

    return -1

def get_targets(target, latitude, longitude, radius):

    url = f'https://www.google.com.br/maps/search/{target}/@{latitude},{longitude},15z/data=!3m1!4b1?entry=ttu'

    response = requests.get(url)
    content = response.content.decode('utf-8')
    content = content.replace('\\\\u0026','&')

    pattern_without_rating = r'((?:-|\+)\d+\.\d+),((?:-|\+)\d+\.\d+)\],\\"0x\w+?:0x\w+?\\",\\\"(.*?)\\"'
    pattern_with_rating = r'([012345]{1}\.\d{1}),(\d+)\](?:,.*?){7}((?:-|\+)\d+\.\d+),((?:-|\+)\d+\.\d+)\],\\"0x\w+?:0x\w+?\\",\\\"(.*?)\\"'

    match_without_rating = re.findall(pattern_without_rating, content)
    match_with_rating = re.findall(pattern_with_rating, content)

    result = []
    for m in match_with_rating + match_without_rating:

        is_duplicate = any(item['nome'] == m[-1] for item in result)

        if not is_duplicate:

            euclidean_distance = get_haversine_calculus(latitude, longitude, float(m[-3]), float(m[-2]))

            if euclidean_distance < radius:

                result.append({
                    'rating': float(m[0]) if len(m) == 5 else 0,
                    'evaluators': int(m[1]) if len(m) == 5 else 0,
                    'latitude': float(m[-3]),
                    'longitude': float(m[-2]),
                    'walking_distance': get_walking_distance(latitude, longitude, float(m[-3]), float(m[-2])),
                    'euclidean_distance': euclidean_distance,
                    'nome': m[-1]
                })

    df_result = pd.DataFrame(result)

    if len(df_result) > 1:

        df_result['walking_distance'] = df_result['walking_distance'].astype(int)

        df_result = df_result[df_result['walking_distance'] <= radius]

        df_result = df_result.sort_values('walking_distance')

    return df_result

In [ ]:
df_adress_enhanced = df_address.copy()

for index, row in df_adress_enhanced.iterrows():

    print(f"{index+1}/{len(df_adress_enhanced)}...")

    if isinstance(row['Latitude'], (int, float)) and math.isnan(row['Latitude']):
        continue

    if isinstance(row['Longitude'], (int, float)) and math.isnan(row['Longitude']):
        continue

    targets = ["restaurante", "farmácia", "academia", "escola", "supermercado", "posto policial", "hotel", "estação de metrô"]

    for target in targets:

        try:
            df_adress_enhanced_tt = get_targets(target, row['Latitude'], row['Longitude'], 1000)
        except Exception as err:
            df_adress_enhanced_tt = pd.DataFrame()
            print(f"Unexpected {err=}, {type(err)=}")

        weighted_mean = 0

        if len(df_adress_enhanced_tt) > 0:

            # Calcule a média ponderada
            weighted_mean = (df_adress_enhanced_tt['rating'] * df_adress_enhanced_tt['evaluators']).sum() / df_adress_enhanced_tt['evaluators'].sum()

        df_adress_enhanced.at[index, f"{target.replace(' ','_')}_average_rating"] = weighted_mean
        df_adress_enhanced.at[index, f"{target.replace(' ','_')}_total"] = len(df_adress_enhanced_tt)

1/28...


<ipython-input-15-8b2dbfd54466>:28: RuntimeWarning: invalid value encountered in scalar divide
  weighted_mean = (df_adress_enhanced_tt['rating'] * df_adress_enhanced_tt['evaluators']).sum() / df_adress_enhanced_tt['evaluators'].sum()


2/28...
3/28...
4/28...
5/28...


KeyboardInterrupt: 

In [ ]:
df_adress_enhanced

# 9. Pré-processamento dos dados

In [ ]:
df_close.rename(columns={'valor': 'Close', 'ticker': 'Ticker', 'data': 'Datetime'}, inplace = True)
df_close['Ticker'] = df_close['Ticker'].astype('category')
df_close['Datetime'] = pd.to_datetime(df_close['Datetime'], format='%d-%m-%YT%H:%M:%S')

In [ ]:
df_dividends = df_dividends[['ticker', 'data_pagamento', 'valor']].copy()
df_dividends.rename(columns={'data_pagamento': 'Datetime', 'ticker': 'Ticker', 'valor': 'Dividends'}, inplace=True)
df_dividends['Ticker'] = df_dividends['Ticker'].astype('category')
df_dividends['Datetime'] = pd.to_datetime(df_dividends['Datetime'], format='%Y-%m-%d')
df_dividends = df_dividends[(df_dividends['Datetime'] >= start_date) & (df_dividends['Datetime'] < end_date)]

In [ ]:
def is_year_month_present(timestamps, year_month):
    year_months = [ts.strftime("%Y-%m") for ts in timestamps]

    return year_month in year_months

def has_missing_data(df_history):

    min_date = df_history['Datetime'].min()
    max_date = df_history['Datetime'].max()

    start_date = pd.to_datetime(max_date).replace(day=1, hour=0, minute=0, second=0, microsecond=0)
    end_date = pd.to_datetime(min_date).replace(day=1, hour=0, minute=0, second=0, microsecond=0)

    while start_date >= end_date:
        year_month = start_date.strftime("%Y-%m")

        if not is_year_month_present(list(df_history['Datetime']), year_month):
            x = list(df_history['Datetime'])
            print(f'{year_month} - {x}')
            return True

        start_date -= relativedelta(months=1)

    return False

def get_month_close(df_close, date):

    date = pd.to_datetime(date)

    year = date.year
    month = date.month

    start_date = pd.to_datetime('today').replace(year=year, month= month, day=1,hour=0,minute=0,second=0,microsecond=0)
    end_date = (start_date + relativedelta(months=1))

    df_aux = df_close.copy()

    df_aux = df_aux[df_aux['Datetime'] >= start_date]
    df_aux = df_aux[df_aux['Datetime'] < end_date]

    if len(df_aux) > 0:
        return float(df_aux.iloc[-1]['Close'])
    else:
        return -1

def get_history(df_close, df_dividends, fund, years):

    df_close_aux = df_close[df_close['Ticker'] == fund]
    df_history = df_dividends[df_dividends['Ticker'] == fund].copy()

    if len(df_history) > 0 and len(df_close_aux) > 0:

        new_df = []
        for index, row in df_history.iterrows():

            row['Dividends'] = round(row['Dividends'],2)
            row['Close'] = get_month_close(df_close_aux, row['Datetime'])
            new_df.append(row)

        df_history = pd.DataFrame(new_df)

        datas = list(df_history['Datetime'])

        if has_missing_data(df_history):
            print("FII " + fund + " será removido por estar com dados faltantes.")
            df_history = pd.DataFrame()

    return df_history

def ajusta_desdobramento(df, df_splits):
    df_splits['Date'] = pd.to_datetime(df_splits['Date'])  # Convertendo a coluna 'Date' para datetime

    for index, row in df.iterrows():
        ticker = row['Ticker']
        date = row['Datetime']
        close_price = row['Close']

        desdobramento = df_splits[df_splits['Ticker'] == ticker]

        if not desdobramento.empty:
            for _, desd_row in desdobramento.iterrows():
                desd_date = desd_row['Date']

                # Extraindo o ano e o mês da data a ser comparada
                date_year_month = date.to_period('M')

                # Extraindo o ano e o mês da data em desd_row
                desd_year_month = desd_date.to_period('M')

                # Verificando se o ano e o mês de `date` são menores do que o ano e o mês de `desd_date`
                if date_year_month < desd_year_month:
                    ratio = desd_row['Ratio']
                    df.at[index, 'Close'] = round(close_price / ratio, 2)

    return df

def remove_big_variations(df, col_dict):
    drop_indexes = []
    for fund in df['Ticker'].unique():
        df_variacoes = df[df['Ticker'] == fund].sort_values(by='Datetime', ascending=True)

        for key in col_dict:
            df_variacoes = df_variacoes[(abs(df_variacoes[key]) >= col_dict[key])]

            if len(df_variacoes) > 0:
                drop_indexes = drop_indexes + list(df[(df["Datetime"] <= df_variacoes["Datetime"].values[-1]) & (df.Ticker == fund)].index)

    df = df.drop(drop_indexes)
    ticker_before = df['Ticker'].unique()

    # Remove funds que não possuem pelo menos 20 registros
    for fund in df['Ticker'].unique():
        if len(df[df['Ticker'] == fund]) < 20:
            df = df[df['Ticker'] != fund]

    removed_tickers = set(ticker_before) - set(df['Ticker'].unique())

    if(len(removed_tickers) > 0):
        print("Tickers removidos: ", removed_tickers)
    else:
        print("Nenhum ticker foi removido")

    return df

def add_pct_changes(df_close):
    # Cria o DataFrame a ser aprimorado
    df_improved = df_close.copy()

    # Remove funds que não possuem pelo menos 12 registros
    for fund in df_improved['Ticker'].unique():
        if len(df_improved[df_improved['Ticker'] == fund]) < 12:
            df_improved = df_improved[df_improved['Ticker'] != fund]

    # Normaliza os dados que sofreram desdobramento
    ajusta_desdobramento(df_improved, df_splits)
    df_improved = df_improved.replace(np.inf, 0).replace(-np.inf, 0).replace(0, 0.001)

    # Cria a coluna DividendYield
    df_improved['DividendYield'] = round(100 * df_improved['Dividends'] / df_improved['Close'], 6)

    # Cria novas colunas contendo a variação de valores ao longo dos meses
    for fund in df_improved['Ticker'].unique():

        print("Incrementando informações de " + fund + "...")

        df_fund = df_improved[df_improved.Ticker == fund].sort_values(by='Datetime', ascending=True)
        df_improved.loc[df_improved.Ticker == fund, 'DividendsChange'] = round(df_fund['Dividends'].pct_change(), 6)
        df_improved.loc[df_improved.Ticker == fund, 'CloseChange'] = round(df_fund['Close'].pct_change(), 6)
        df_improved.loc[df_improved.Ticker == fund, 'DividendYieldChange'] = round(df_fund['DividendYield'].pct_change(), 6)
        df_improved.loc[df_improved.Ticker == fund, 'DividendsChange6M'] = round(df_fund['Dividends'].pct_change(periods=6), 6)
        df_improved.loc[df_improved.Ticker == fund, 'CloseChange6M'] = round(df_fund['Close'].pct_change(periods=6), 6)
        df_improved.loc[df_improved.Ticker == fund, 'DividendYieldChange6M'] = round(df_fund['DividendYield'].pct_change(periods=6), 6)

    # Procura no DataFrame registros com variações muito discrepantes
    df_improved = remove_big_variations(df_improved, {'DividendsChange': 500, 'CloseChange': 0.35})

    return df_improved

def process_history(df_funds, years):

    df_history = pd.DataFrame()

    # Percorre a lista de funds para obter o histórico individual de cada um deles
    for fund in df_funds['ticker']:

        print("Coletando informações de " + fund + "...")

        df_aux = get_history(df_close, df_dividends, fund, years)
        df_history = pd.concat([df_history, df_aux], ignore_index=True)

        print(str(len(df_aux)) + " dados de histórico foram encontrados.")

    if df_history.empty:
        print("O DataFrame está vazio.")
        return df_history

    # Tratar valores NaN
    is_NaN = df_history.isnull()
    row_has_NaN = is_NaN.any(axis=1)
    rows_with_NaN = df_history[row_has_NaN]

    if not rows_with_NaN.empty:
        tickers = rows_with_NaN['Ticker'].unique()
        df_history = df_history[~df_history['Ticker'].isin(tickers)]

    # Remover duplicatas e substituir valores infinitos e zero
    df_history = df_history.drop_duplicates().replace([np.inf, -np.inf, 0], 0.001)

    # Verificar a quantidade de registros por fund
    for fund in df_history['Ticker'].unique():
        if len(df_history[df_history['Ticker'] == fund]) < 12:
            df_history = df_history[df_history['Ticker'] != fund]

    # Comparar tickers no último mês
    if 'ticker' in df_history.columns and 'Datetime' in df_history.columns:

        target_date = pd.to_datetime(end_date, format='%d-%m-%Y')

        # Extraindo o ano e o mês de end_date
        target_year = target_date.year
        target_month = target_date.month

        # Filtre os valores que correspondem ao mês e ano desejados
        a = df_history[(df_history['Datetime'].dt.year == target_year) & (df_history['Datetime'].dt.month == target_month)]
        b = df_history.ticker.unique()
        intersection = list(set(a) & set(b))
        funds_faltantes = list(set(a) ^ set(b))

        df_history = df_history[~df_history['Ticker'].isin(funds_faltantes)]

    return df_history

def add_indices(df_history):
    # Convertendo colunas de datas para datetime
    df_ifix['Datetime'] = pd.to_datetime(df_ifix['Datetime'])
    df_indices['Timestamp'] = pd.to_datetime(df_indices['Timestamp'], format='%Y-%m')

    # Criando colunas YYYY-MM nos DataFrames
    df_ifix['YYYY-MM'] = df_ifix['Datetime'].dt.to_period('M')
    df_indices['YYYY-MM'] = df_indices['Timestamp'].dt.to_period('M')
    df_history['YYYY-MM'] = df_history['Datetime'].dt.to_period('M')

    # Obter o último valor de df_ifix para cada YYYY-MM
    # Obter o último valor de df_ifix para cada YYYY-MM
    df_ifix_last = df_ifix.sort_values(by='Datetime').groupby('YYYY-MM').last().reset_index()

    # Renomeando coluna Close para IFIX antes da união
    df_ifix_last.rename(columns={'Close': 'IFIX'}, inplace=True)

    # Pivotando df_indices para ter uma coluna para cada índice
    df_indices_pivot = df_indices.pivot(index='YYYY-MM', columns='Indice', values='Value').reset_index()

    # Unindo df_history com df_ifix_last
    df_history = df_history.merge(df_ifix_last[['YYYY-MM', 'IFIX']], on='YYYY-MM', how='left')

    # Unindo df_history com df_indices_pivot
    df_history = df_history.merge(df_indices_pivot, on='YYYY-MM', how='left')

    # Removendo coluna temporária YYYY-MM
    df_history.drop(columns=['YYYY-MM'], inplace=True)

    df_history.rename(columns={'IGP-M': 'IGPM'}, inplace=True)

    return df_history

def getSectorMeans(df_funds, df_history):

    df_setores = []

    for setor in df_funds["setor"].unique():
        setor_tickers = df_funds[df_funds["setor"] == setor]["ticker"].values
        df_sector = df_history[df_history["Ticker"].isin(setor_tickers)]

        min_date = pd.to_datetime(df_sector["Datetime"].min()).replace(day=1)
        max_date = pd.to_datetime(df_sector["Datetime"].max()).replace(day=1)

        while min_date <= max_date:
            date = min_date.strftime("%Y-%m")

            setor_data = {
                'setor': setor,
                'Datetime': date,
                'DividendsChangeMean': df_sector[df_sector["Datetime"].dt.strftime("%Y-%m") == date]["DividendsChange"].mean(),
                'CloseChangeMean': df_sector[df_sector["Datetime"].dt.strftime("%Y-%m") == date]["CloseChange"].mean(),
                'DividendYieldChangeMean': df_sector[df_sector["Datetime"].dt.strftime("%Y-%m") == date]["DividendYieldChange"].mean(),
                'DividendsChangeMean6M': df_sector[df_sector["Datetime"].dt.strftime("%Y-%m") == date]["DividendsChange6M"].mean(),
                'CloseChangeMean6M': df_sector[df_sector["Datetime"].dt.strftime("%Y-%m") == date]["CloseChange6M"].mean(),
                'DividendYieldChangeMean6M': df_sector[df_sector["Datetime"].dt.strftime("%Y-%m") == date]["DividendYieldChange6M"].mean(),
            }

            df_setores.append(setor_data)
            min_date += relativedelta(months=1)

    return pd.DataFrame(df_setores)


def add_sector_pct_changes(df_funds, df_history):

    df_sectors = getSectorMeans(df_funds, df_history)

    # Insere preço dos índices e a média do setor ao longo do tempo
    for index, fundo in enumerate(df_history['Ticker'].unique()):

        print(str(index+1) + "/" + str(len(df_history['Ticker'].unique())))

        sector = df_funds[df_funds["ticker"] == fundo]["setor"].values[0]

        for data in df_history['Datetime']:

            sector_values = df_sectors[(df_sectors["Datetime"] == data.strftime("%Y-%m")) & (df_sectors["setor"] == sector)]

            if len(sector_values) > 0:
                for mean_col in ["DividendsChangeMean", "CloseChangeMean", "DividendYieldChangeMean"]:
                    mean_value = sector_values[mean_col].values[0]
                    df_history.loc[(df_history.Ticker == fundo) & (df_history.Datetime == data), "Sector" + mean_col] = float(mean_value)

    return df_history

In [ ]:
df_history = process_history(df_funds, 2)

Coletando informações de BCFF11...
36 dados de histórico foram encontrados.
Coletando informações de BTLG11...
36 dados de histórico foram encontrados.
Coletando informações de CPTS11...
36 dados de histórico foram encontrados.
Coletando informações de HGLG11...
36 dados de histórico foram encontrados.
Coletando informações de HGRU11...
36 dados de histórico foram encontrados.
Coletando informações de IRDM11...
36 dados de histórico foram encontrados.
Coletando informações de KNCR11...
36 dados de histórico foram encontrados.
Coletando informações de KNRI11...
36 dados de histórico foram encontrados.
Coletando informações de MXRF11...
36 dados de histórico foram encontrados.
Coletando informações de VGHF11...
33 dados de histórico foram encontrados.
Coletando informações de VGIR11...
36 dados de histórico foram encontrados.
Coletando informações de VISC11...
36 dados de histórico foram encontrados.
Coletando informações de XPLG11...
36 dados de histórico foram encontrados.
Coletando in

In [ ]:
df_history = add_pct_changes(df_history)

Incrementando informações de BCFF11...
Incrementando informações de BTLG11...
Incrementando informações de CPTS11...
Incrementando informações de HGLG11...
Incrementando informações de HGRU11...
Incrementando informações de IRDM11...
Incrementando informações de KNCR11...
Incrementando informações de KNRI11...
Incrementando informações de MXRF11...
Incrementando informações de VGHF11...
Incrementando informações de VGIR11...
Incrementando informações de VISC11...
Incrementando informações de XPLG11...
Incrementando informações de XPML11...
tickers removidos:  set()


In [ ]:
df_history = add_indices(df_history)

In [ ]:
df_history = add_sector_pct_changes(df_funds, df_history)

1/14
2/14
3/14
4/14
5/14
6/14
7/14
8/14
9/14
10/14
11/14
12/14
13/14
14/14


In [ ]:
df_history['Datetime'] = pd.to_datetime(df_history['Datetime']).dt.to_period('M')
df_history = df_history.dropna(subset=['DividendsChange'])
df_history

,Ticker,Datetime,Dividends,Close,DividendYield,DividendsChange,CloseChange,DividendYieldChange,DividendsChange6M,CloseChange6M,DividendYieldChange6M,IFIX,IGPM,IPCA,SELIC,SectorDividendsChangeMean,SectorCloseChangeMean,SectorDividendYieldChangeMean
0,BCFF11,2023-12,0.07,9.37,0.747065,0.000000,0.035359,-0.034152,0.0,0.045759,-0.043757,3.311,0.74,0.56,12.15,0.000000,0.035359,-0.034152
1,BCFF11,2023-11,0.07,9.05,0.773481,0.000000,0.051103,-0.048618,0.0,0.072275,-0.067403,3.177,0.59,0.28,12.65,0.000000,0.051103,-0.048618
2,BCFF11,2023-10,0.07,8.61,0.813008,0.000000,-0.055921,0.059233,0.0,0.016529,-0.016260,3.156,0.50,0.24,12.65,0.000000,-0.055921,0.059233
3,BCFF11,2023-09,0.07,9.12,0.767544,0.000000,-0.016181,0.016448,0.0,0.179819,-0.152412,3.219,0.37,0.26,13.15,0.000000,-0.016181,0.016448
4,BCFF11,2023-08,0.07,9.27,0.755124,0.000000,0.026578,-0.025890,0.0,0.197674,-0.165049,3.213,-0.14,0.23,13.65,0.000000,0.026578,-0.025890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,XPML11,2021-06,0.40,103.20,0.387597,0.333333,-0.012629,0.350390,NaN,NaN,NaN,NaN,0.60,0.53,3.40,0.366667,-0.015984,0.388998
496,XPML11,2021-05,0.30,104.52,0.287026,1.000000,-0.013962,1.028323,NaN,NaN,NaN,NaN,4.10,0.83,2.65,0.694445,-0.019883,0.727002
497,XPML11,2021-04,0.15,106.00,0.141509,-0.400000,0.002838,-0.401699,NaN,NaN,NaN,NaN,1.51,0.31,2.65,-0.536363,0.008987,-0.539653
498,XPML11,2021-03,0.25,105.70,0.236518,-0.285714,-0.024818,-0.267538,NaN,NaN,NaN,NaN,2.94,0.93,1.90,-0.184524,-0.030618,-0.158112
